# Academy of Py
GWU Data Analytics Bootcamp Homework 4

### Observable Trends

* Students tend to score higher and are more likely to pass in Reading than in Math
* Students who attend Charter schools tend to score higher and are more likely to pass than students who attend District Schools
* Students who attend small schools tend to score higher and are more likely to pass than students who attend larger schools

In [3]:
# Load dependencies

import pandas as pd
import numpy as np

In [4]:
# Find data

school_csv = "../Resources/schools_complete.csv"
student_csv = "../Resources/students_complete.csv"

In [5]:
# Read in data and create dataframes

school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

In [6]:
# Merge initial dataframes

school_df = school_df.rename(columns={"name":"school"})
df = pd.merge(school_df, student_df, on="school")

### District Summary

In [9]:
# Set baseline for minimum passing grade

min_passing_grade = 70

# Calculate summary statistics (Overall Passing Rate is calculated as average of Math and Reading passing rates)

num_of_schools = df['School ID'].nunique()
num_of_students = df['Student ID'].nunique()
tot_school_budget = school_df['budget'].sum()
avg_math_score_tot = df['math_score'].mean()
avg_reading_score_tot = df['reading_score'].mean()
percent_passing_math_tot = df.loc[df['math_score'] >= min_passing_grade, :]['math_score'].count() / num_of_students
percent_passing_reading_tot = df.loc[df['reading_score'] >= min_passing_grade, :]['reading_score'].count() / num_of_students
overall_passing_score_tot = (percent_passing_math_tot + percent_passing_reading_tot) / 2

# Create and format summary statistics dataframe

district_summary_df = pd.DataFrame({"Total Schools":[num_of_schools], "Total Students":[num_of_students], "Total Budget":[tot_school_budget], "Average Math Score":[avg_math_score_tot], "Average Reading Score":[avg_reading_score_tot], "% Passing Math":[percent_passing_math_tot], "% Passing Reading":[percent_passing_reading_tot], "% Overall Passing Rate":[overall_passing_score_tot]})
district_summary_df = district_summary_df[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# Format data

district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map("${:,}".format)
district_summary_df['Average Math Score'] = district_summary_df['Average Math Score'].map("{:.2f}".format)
district_summary_df['Average Reading Score'] = district_summary_df['Average Reading Score'].map("{:.2f}".format)
district_summary_df['% Passing Math'] = (district_summary_df['% Passing Math'] * 100).map("{:.2f}%".format)
district_summary_df['% Passing Reading'] = (district_summary_df['% Passing Reading'] * 100).map("{:.2f}%".format)
district_summary_df['% Overall Passing Rate'] = (district_summary_df['% Overall Passing Rate'] * 100).map("{:.2f}%".format)

# Print data

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428",78.99,81.88,74.98%,85.81%,80.39%


### School Summary

In [88]:
# Group and aggregate data

grouped_schools_funcs = df.groupby(['school']).agg({'Student ID':['count'], 'reading_score':['mean'], "math_score":['mean']})
grouped_schools_funcs = grouped_schools_funcs.reset_index()
grouped_schools_funcs.columns = ['School Name', 'Number of Students', 'Avg Reading Score', 'Avg Math Score']

# Add additional columns and calculate Per Student Budget

short_school_df = school_df[['school', 'type', 'budget']]
short_school_df.columns = ['School Name', 'School Type', 'Budget']
school_summary_df = pd.merge(short_school_df, grouped_schools_funcs, on='School Name')
school_summary_df['Per Student Budget'] = school_summary_df['Budget'] / school_summary_df['Number of Students']

# Calculate Percent Passing Reading

df_passing_reading = df.loc[df['reading_score'] >= min_passing_grade, :]
df_passing_reading_by_school = df_passing_reading.groupby('school')
percent_passing_reading_by_school = df_passing_reading_by_school['reading_score'].count() / df.groupby('school')['reading_score'].count()
percent_passing_reading_by_school_df = percent_passing_reading_by_school.reset_index()
percent_passing_reading_by_school_df.columns = ['School Name', '% Passing Reading']

# Calculate Percent Passing Math

df_passing_math = df.loc[df['math_score'] >= min_passing_grade, :]
df_passing_math_by_school = df_passing_math.groupby('school')
percent_passing_math_by_school = df_passing_math_by_school['math_score'].count() / df.groupby('school')['math_score'].count()
percent_passing_math_by_school_df = percent_passing_math_by_school.reset_index()
percent_passing_math_by_school_df.columns = ['School Name', '% Passing Math']

# Combine dataframes and calculate Overall Passing Rate

percent_passing_by_school_df = pd.merge(percent_passing_reading_by_school_df, percent_passing_math_by_school_df, on="School Name")
merged_school_summary_df = pd.merge(school_summary_df, percent_passing_by_school_df, on="School Name")
merged_school_summary_df['% Overall Passing Rate'] = (merged_school_summary_df['% Passing Reading'] + merged_school_summary_df['% Passing Math']) / 2

# Format dataframe

merged_school_summary_df = merged_school_summary_df.set_index('School Name')
merged_school_summary_df = merged_school_summary_df[['School Type', 'Number of Students', 'Budget', 'Per Student Budget', 'Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
merged_school_summary_df.columns = ['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']

# Format data

merged_school_summary_df['Total School Budget'] = merged_school_summary_df['Total School Budget'].map("${:,}".format)
merged_school_summary_df['Per Student Budget'] = merged_school_summary_df['Per Student Budget'].map("${:,.0f}".format)
merged_school_summary_df['Average Math Score'] = merged_school_summary_df['Average Math Score'].map("{:.2f}".format)
merged_school_summary_df['Average Reading Score'] = merged_school_summary_df['Average Reading Score'].map("{:.2f}".format)
merged_school_summary_df['% Passing Math'] = (merged_school_summary_df['% Passing Math'] * 100).map("{:.2f}%".format)
merged_school_summary_df['% Passing Reading'] = (merged_school_summary_df['% Passing Reading'] * 100).map("{:.2f}%".format)
merged_school_summary_df['% Overall Passing Rate'] = (merged_school_summary_df['% Overall Passing Rate'] * 100).map("{:.2f}%".format)

# Print data

merged_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,73.50%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,73.36%
Shelton High School,Charter,1761,"$1,056,600",$600,83.36,83.73,93.87%,95.85%,94.86%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,73.81%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,2283,"$1,319,574",$578,83.27,83.99,93.87%,96.54%,95.20%
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,95.59%
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,66.68%,81.93%,74.31%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,94.38%


### Top Performing Schools (By Passing Rate)

In [89]:
# Isolate Top 5 schools by Overall Passing Rate

top_schools = merged_school_summary_df.sort_values('% Overall Passing Rate', ascending=False).head()
top_schools = top_schools[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,"$1,043,130",$638,83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,2283,"$1,319,574",$578,83.27,83.99,93.87%,96.54%,95.20%


### Bottom Performing Schools (By Passing Rate)

In [90]:
# Isolate Bottom 5 schools by Overall Passing Rate

bottom_schools = merged_school_summary_df.sort_values('% Overall Passing Rate').head()
bottom_schools = bottom_schools[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,68.31%,79.30%,73.80%


### Math Scores by Grade

In [91]:
# Filter data by grade level

grade_9_df = df.loc[df['grade'] == '9th', :]
grade_10_df = df.loc[df['grade'] == '10th', :]
grade_11_df = df.loc[df['grade'] == '11th', :]
grade_12_df = df.loc[df['grade'] == '12th', :]

# Calculate Average Math Scores

math_by_grade_df = grade_9_df.groupby(['school']).agg({'math_score':['mean']})
math_by_grade_df['10th'] = grade_10_df.groupby(['school']).agg({'math_score':['mean']})
math_by_grade_df['11th'] = grade_11_df.groupby(['school']).agg({'math_score':['mean']})
math_by_grade_df['12th'] = grade_12_df.groupby(['school']).agg({'math_score':['mean']})

# Format dataframe

math_by_grade_df.columns = ['9th', '10th', '11th', '12th']

# Format data

math_by_grade_df['9th'] = math_by_grade_df['9th'].map("{:.2f}".format)
math_by_grade_df['10th'] = math_by_grade_df['10th'].map("{:.2f}".format)
math_by_grade_df['11th'] = math_by_grade_df['11th'].map("{:.2f}".format)
math_by_grade_df['12th'] = math_by_grade_df['12th'].map("{:.2f}".format)

# Print Data

math_by_grade_df

,9th,10th,11th,12th
school,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


### Reading Scores by Grade

In [92]:
# Calculate Average Reading Scores by grade for each school

reading_by_grade_df = grade_9_df.groupby(['school']).agg({'reading_score':['mean']})
reading_by_grade_df['10th'] = grade_10_df.groupby(['school']).agg({'reading_score':['mean']})
reading_by_grade_df['11th'] = grade_11_df.groupby(['school']).agg({'reading_score':['mean']})
reading_by_grade_df['12th'] = grade_12_df.groupby(['school']).agg({'reading_score':['mean']})

# Format dataframe

reading_by_grade_df.columns = ['9th', '10th', '11th', '12th']

# Format data

reading_by_grade_df['9th'] = reading_by_grade_df['9th'].map("{:.2f}".format)
reading_by_grade_df['10th'] = reading_by_grade_df['10th'].map("{:.2f}".format)
reading_by_grade_df['11th'] = reading_by_grade_df['11th'].map("{:.2f}".format)
reading_by_grade_df['12th'] = reading_by_grade_df['12th'].map("{:.2f}".format)

# Print Data

reading_by_grade_df

,9th,10th,11th,12th
school,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


### Scores by School Spending

In [94]:
# Create and assign Per Student Budget bins

school_spending_df = df
bins = [570, 599, 629, 649, 659]
group_names = ['< $600', '$600-629', '$630-649', '$650-659']
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Spending"], bins, labels=group_names)

# Group and aggregate data

school_spending_df_grouped = school_spending_df.groupby('Spending Ranges (Per Student)').agg({'reading_score':['mean'], 'math_score':['mean']})
school_spending_summary_df = school_spending_df_grouped.reset_index()
school_spending_summary_df.columns = ['Spending Ranges (Per Student)', 'Average Reading Score', 'Average Math Score']

# Calculate Percent Passing Reading

passing_reading_by_budget = school_spending_df.loc[school_spending_df['reading_score'] >= min_passing_grade, :]
df_passing_reading_by_budget = passing_reading_by_budget.groupby('Spending Ranges (Per Student)')
percent_passing_reading_by_budget = df_passing_reading_by_budget['reading_score'].count() / school_spending_df.groupby('Spending Ranges (Per Student)')['reading_score'].count()
percent_passing_reading_by_budget_df = percent_passing_reading_by_budget.reset_index()
percent_passing_reading_by_budget_df.columns = ['Spending Ranges (Per Student)', '% Passing Reading']

# Calculate Percent Passing Math

passing_math_by_budget = school_spending_df.loc[school_spending_df['math_score'] >= min_passing_grade, :]
df_passing_math_by_budget = passing_math_by_budget.groupby('Spending Ranges (Per Student)')
percent_passing_math_by_budget = df_passing_math_by_budget['math_score'].count() / school_spending_df.groupby('Spending Ranges (Per Student)')['math_score'].count()
percent_passing_math_by_budget_df = percent_passing_math_by_budget.reset_index()
percent_passing_math_by_budget_df.columns = ['Spending Ranges (Per Student)', '% Passing Math']

# Combine dataframes and calculate Overall Passing Rate

percent_passing_by_budget_df = pd.merge(percent_passing_reading_by_budget_df, percent_passing_math_by_budget_df, on="Spending Ranges (Per Student)")
merged_spending_summary_df = pd.merge(school_spending_summary_df, percent_passing_by_budget_df, on="Spending Ranges (Per Student)")
merged_spending_summary_df['% Overall Passing Rate'] = (merged_spending_summary_df['% Passing Reading'] + merged_spending_summary_df['% Passing Math']) / 2

# Format dataframe

merged_spending_summary_df = merged_spending_summary_df.set_index('Spending Ranges (Per Student)')
merged_spending_summary_df = merged_spending_summary_df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# Format data

merged_spending_summary_df['Average Math Score'] = merged_spending_summary_df['Average Math Score'].map("{:.2f}".format)
merged_spending_summary_df['Average Reading Score'] = merged_spending_summary_df['Average Reading Score'].map("{:.2f}".format)
merged_spending_summary_df['% Passing Math'] = (merged_spending_summary_df['% Passing Math'] * 100).map("{:.2f}%".format)
merged_spending_summary_df['% Passing Reading'] = (merged_spending_summary_df['% Passing Reading'] * 100).map("{:.2f}%".format)
merged_spending_summary_df['% Overall Passing Rate'] = (merged_spending_summary_df['% Overall Passing Rate'] * 100).map("{:.2f}%".format)

# Print Data

merged_spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
< $600,83.36,83.96,93.70%,96.69%,95.19%
$600-629,79.98,82.31,79.11%,88.51%,83.81%
$630-649,77.82,81.30,70.62%,82.60%,76.61%
$650-659,77.05,81.01,66.23%,81.11%,73.67%


### Scores by School Size

In [95]:
# Create and assign school size bins

school_size_df = df
bins = [0, 1999, 2999, 4999]
group_names = ['Small (< 2000)', 'Medium (2000-2999)', 'Large (3000+)']
school_size_df["School Size"] = pd.cut(school_size_df["size"], bins, labels=group_names)

# Group and aggregate data

school_size_df_grouped = school_size_df.groupby('School Size').agg({'reading_score':['mean'], 'math_score':['mean']})
school_size_summary_df = school_size_df_grouped.reset_index()
school_size_summary_df.columns = ['School Size', 'Average Reading Score', 'Average Math Score']

# Calculate Percent Passing Reading

passing_reading_by_size = school_size_df.loc[school_size_df['reading_score'] >= min_passing_grade, :]
df_passing_reading_by_size = passing_reading_by_size.groupby('School Size')
percent_passing_reading_by_size = df_passing_reading_by_size['reading_score'].count() / school_size_df.groupby('School Size')['reading_score'].count()
percent_passing_reading_by_size_df = percent_passing_reading_by_size.reset_index()
percent_passing_reading_by_size_df.columns = ['School Size', '% Passing Reading']

# Calculate Percent Passing Math

passing_math_by_size = school_size_df.loc[school_size_df['math_score'] >= min_passing_grade, :]
df_passing_math_by_size = passing_math_by_size.groupby('School Size')
percent_passing_math_by_size = df_passing_math_by_size['math_score'].count() / school_size_df.groupby('School Size')['math_score'].count()
percent_passing_math_by_size_df = percent_passing_math_by_size.reset_index()
percent_passing_math_by_size_df.columns = ['School Size', '% Passing Math']

# Combine dataframes and calculate Overall Passing Rate

percent_passing_by_size_df = pd.merge(percent_passing_reading_by_size_df, percent_passing_math_by_size_df, on="School Size")
merged_size_summary_df = pd.merge(school_size_summary_df, percent_passing_by_size_df, on="School Size")
merged_size_summary_df['% Overall Passing Rate'] = (merged_size_summary_df['% Passing Reading'] + merged_size_summary_df['% Passing Math']) / 2

# Format dataframe

merged_size_summary_df = merged_size_summary_df.set_index('School Size')
merged_size_summary_df = merged_size_summary_df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# Format data

merged_size_summary_df['Average Math Score'] = merged_size_summary_df['Average Math Score'].map("{:.2f}".format)
merged_size_summary_df['Average Reading Score'] = merged_size_summary_df['Average Reading Score'].map("{:.2f}".format)
merged_size_summary_df['% Passing Math'] = (merged_size_summary_df['% Passing Math'] * 100).map("{:.2f}%".format)
merged_size_summary_df['% Passing Reading'] = (merged_size_summary_df['% Passing Reading'] * 100).map("{:.2f}%".format)
merged_size_summary_df['% Overall Passing Rate'] = (merged_size_summary_df['% Overall Passing Rate'] * 100).map("{:.2f}%".format)

# Print data

merged_size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (< 2000),83.44,83.88,93.66%,96.67%,95.17%
Medium (2000-2999),78.16,81.65,72.34%,83.84%,78.09%
Large (3000+),77.07,80.93,66.47%,81.11%,73.79%


### Scores by School Type

In [10]:
# Group and aggregate data by School Type

type_summary = df.groupby('type').agg({'reading_score':['mean'], 'math_score':['mean']})
type_summary_df = type_summary.reset_index()
type_summary_df.columns = ['School Type', 'Average Reading Score', 'Average Math Score']

# Calculate Percent Passing Reading

df_passing_reading = df.loc[df['reading_score'] >= min_passing_grade, :]
df_passing_reading_by_type = df_passing_reading.groupby('type')
percent_passing_reading_by_type = df_passing_reading_by_type['reading_score'].count() / df.groupby('type')['reading_score'].count()
percent_passing_reading_by_type_df = percent_passing_reading_by_type.reset_index()
percent_passing_reading_by_type_df.columns = ['School Type', '% Passing Reading']

# Calculate Percent Passing Math

df_passing_math = df.loc[df['math_score'] >= min_passing_grade, :]
df_passing_math_by_type = df_passing_math.groupby('type')
percent_passing_math_by_type = df_passing_math_by_type['math_score'].count() / df.groupby('type')['math_score'].count()
percent_passing_math_by_type_df = percent_passing_math_by_type.reset_index()
percent_passing_math_by_type_df.columns = ['School Type', '% Passing Math']

# Combine dataframes and calculate Overall Passing Rate

percent_passing_by_type_df = pd.merge(percent_passing_reading_by_type_df, percent_passing_math_by_type_df, on="School Type")
merged_type_summary_df = pd.merge(type_summary_df, percent_passing_by_type_df, on="School Type")
merged_type_summary_df['% Overall Passing Rate'] = (merged_type_summary_df['% Passing Reading'] + merged_type_summary_df['% Passing Math']) / 2

# Format dataframe

merged_type_summary_df = merged_type_summary_df.set_index('School Type')
merged_type_summary_df = merged_type_summary_df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# Format data

merged_type_summary_df['Average Math Score'] = merged_type_summary_df['Average Math Score'].map("{:.2f}".format)
merged_type_summary_df['Average Reading Score'] = merged_type_summary_df['Average Reading Score'].map("{:.2f}".format)
merged_type_summary_df['% Passing Math'] = (merged_type_summary_df['% Passing Math'] * 100).map("{:.2f}%".format)
merged_type_summary_df['% Passing Reading'] = (merged_type_summary_df['% Passing Reading'] * 100).map("{:.2f}%".format)
merged_type_summary_df['% Overall Passing Rate'] = (merged_type_summary_df['% Overall Passing Rate'] * 100).map("{:.2f}%".format)

# Print data

merged_type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%
